In [1]:
import omop_alchemy as oa
from omop_alchemy.db import Base, engine
from omop_alchemy.helpers import create_db, populate_demo_db, to_load_vocabulary, to_load_clinical, to_load_health_system

In [2]:
# this script will create a new sqlite db file with the tables to reflect the specified objects in 
# clinical and reference directories. needs to be made more friendly for re-create / update etc, but 
# will do for now :)

create_db(Base, engine)

2024-02-05 13:58:22,491 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-05 13:58:22,492 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("condition_occurrence")
2024-02-05 13:58:22,492 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-02-05 13:58:22,494 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("condition_occurrence")
2024-02-05 13:58:22,495 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-02-05 13:58:22,496 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("device_exposure")
2024-02-05 13:58:22,496 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-02-05 13:58:22,497 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("device_exposure")
2024-02-05 13:58:22,498 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-02-05 13:58:22,499 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("drug_exposure")
2024-02-05 13:58:22,499 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-02-05 13:58:22,500 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("drug_exposure")
2024-02-05 1

In [3]:
# make sure to download a small athena zip file (suggested no more than 2-4 vocabs for demo purposes)
# and unzip to omop_alchemy/resources/ohdsi_vocabs before running this script. 

populate_demo_db(to_load_health_system)

/Users/georginakennedy/cloudstor/CBDRH/ACDN/OMOP_Alchemy/omop_alchemy/resources/demo_data
complete load for: CARE_SITE.csv
complete load for: LOCATION.csv
complete load for: PROVIDER.csv
2024-02-05 13:58:23,613 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-05 13:58:23,615 INFO sqlalchemy.engine.Engine INSERT INTO location (location_id, address_1, address_2, city, state, zip_code, county, country, location_source_value, latitude, longitude) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
2024-02-05 13:58:23,616 INFO sqlalchemy.engine.Engine [generated in 0.00074s] [(1, 'Liverpool Hospital', 'Burnside Dr', 'Sydney', 'NSW', '2170', 'Liverpool', 'Australia', '', 0.0, 0.0), (2, 'Bankstown Lidcombe Hospital', 'Eldridge Rd', 'Sydney', 'NSW', '2200', 'Bankstown', 'Australia', '', 0.0, 0.0), (3, 'Campbelltown Hospital', 'Off Parkside Cres', 'Sydney', 'NSW', '2560', 'Campbelltown', 'Australia', '', 0.0, 0.0)]
2024-02-05 13:58:23,618 INFO sqlalchemy.engine.Engine INSERT INTO care_site (care_sit

In [4]:

populate_demo_db(to_load_clinical)

/Users/georginakennedy/cloudstor/CBDRH/ACDN/OMOP_Alchemy/omop_alchemy/resources/demo_data
complete load for: PERSON.csv
2024-02-05 13:58:24,269 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-05 13:58:24,277 INFO sqlalchemy.engine.Engine INSERT INTO person (person_id, birth_datetime, death_datetime, person_source_value, gender_source_value, race_source_value, ethnicity_source_value, year_of_birth, month_of_birth, day_of_birth, location_id, provider_id, care_site_id, gender_source_concept_id, ethnicity_source_concept_id, race_source_concept_id, gender_concept_id, ethnicity_concept_id, race_concept_id) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
2024-02-05 13:58:24,283 INFO sqlalchemy.engine.Engine [generated in 0.00646s] [(1, None, None, '1234', 'M', '', '', 1950, 1, 10, 1, 1, 1, 0, 0, 0, 8507, 0, 0), (2, None, None, '2354', 'F', '', '', 1960, 2, 11, 1, 2, 1, 0, 0, 0, 8532, 0, 0), (3, '1970-03-12 00:00:00.000000', None, '2255', 'M', '', '', 1970, 3, 12, 3, 9,